# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('jeopardy.csv')

In [ ]:
df

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [7]:
df.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [3]:
def filter_questions_by_keywords(df, column_name, search_strings):
    """
    Filters rows in a DataFrame where the specified column contains any of the search strings.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to filter.
    column_name (str): The name of the column to search within.
    search_strings (list): A list of strings to search for.
    
    Returns:
    pd.Series: A Series containing the filtered rows.
    """
    # Combine the list into a single regex pattern with word boundaries and optional possessive
    pattern = r'\b(?:' + '|'.join(search_strings) + r")(?:'s)?\b"
    
    # Select rows where the specified column contains any of the search strings
    filtered_df = df[df[column_name].str.contains(pattern, case=False, regex=True)]
    
    return filtered_df

In [22]:
search_strings = ['England']
filtered_questions = filter_questions_by_keywords(df, ' Question', search_strings)
print(filtered_questions)

116                                               She was born in Virginia around 1596 & died in Kent, England in 1617
146                    After a 15-year stay in England, this proprietor of Pennsylvania returned to his colony in 1699
296                           On December 27, 1831 it departed Plymouth, England to map the coastline of South America
386                             The Tanglewood Music Festival is a summer highlight in Lenox in this New England state
621                                                 Famous landmark composed of chalk in the county of Kent in England
                                                              ...                                                     
215892                                                                    New England College & Daniel Webster College
216021                His nickname was "Bertie", but he used this name & number when he became king of England in 1901
216035            In 1699 Capt. Kidd was out pir

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [4]:
df[' Value'].head(10)

0    $200
1    $200
2    $200
3    $200
4    $200
5    $200
6    $400
7    $400
8    $400
9    $400
Name:  Value, dtype: object

In [ ]:
set(df[~df[' Value'].str.contains(r'\$', regex=True)][' Value'])

In [8]:
df[' Value'] = df[' Value'].str.replace(r'\$', '', regex=True)

In [13]:
df[' Value'] = df[' Value'].apply(lambda x: None if x == 'no value' else float(x.replace(',', '')))

In [28]:
df[' Value'].dropna().value_counts()

 Value
400.0      42244
800.0      31860
200.0      30455
1000.0     21640
600.0      20377
           ...  
5401.0         1
1183.0         1
1203.0         1
11600.0        1
11200.0        1
Name: count, Length: 145, dtype: int64

In [26]:
search_strings = ['King']
filtered_questions = filter_questions_by_keywords(df, ' Question', search_strings)
filtered_questions[' Value'].mean()

817.6001483679526

5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [4]:
import string

# Function to convert to lowercase and remove punctuation
def to_lowercase_and_remove_punctuation(text):
    return text.lower().translate(str.maketrans('', '', string.punctuation))

In [33]:
answer_counts = filtered_questions[' Answer'].apply(to_lowercase_and_remove_punctuation).value_counts()
answer_counts

6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?

In [36]:
df[' Air Date'] = pd.to_datetime(df[' Air Date'])
df[' Air Year'] = df[' Air Date'].dt.year

In [39]:
filtered_questions = filter_questions_by_keywords(df, ' Question', ['Computer'])
filtered_questions['Decade'] = (filtered_questions[' Air Year'] // 10) * 10
filtered_questions['Decade'].value_counts().sort_index()

C:\Users\Dave\AppData\Local\Temp\ipykernel_6320\407787948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_questions['Decade'] = (filtered_questions[' Air Year'] // 10) * 10


Decade
1980      5
1990     79
2000    227
2010     51
Name: count, dtype: int64

Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?

In [46]:
category_round = df.groupby([' Category', ' Round']).size().reset_index(name='counts')
category_round_pivot = category_round.pivot(index=' Category', columns=' Round', values='counts').fillna(0)
category_round_pivot

Round,Double Jeopardy!,Final Jeopardy!,Jeopardy!,Tiebreaker
Category,,,,
A JIM CARREY FILM FESTIVAL,0.0,0.0,5.0,0.0
"""!""",0.0,0.0,5.0,0.0
"""-ARES""",5.0,0.0,0.0,0.0
"""-ICIAN"" EXPEDITION",0.0,0.0,5.0,0.0
"""...OD"" WORDS",5.0,0.0,0.0,0.0
...,...,...,...,...
“R” MOVIES,5.0,0.0,0.0,0.0
“SAINTS”,4.0,0.0,0.0,0.0
“SOUTH”,5.0,0.0,0.0,0.0


In [47]:
category_round_pivot['Top Round'] = category_round_pivot.idxmax(axis=1)

In [50]:
category_round_pivot.loc['LITERATURE']

 Round
Double Jeopardy!               381.0
Final Jeopardy!                 10.0
Jeopardy!                      105.0
Tiebreaker                       0.0
Top Round           Double Jeopardy!
Name: LITERATURE, dtype: object

Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [5]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

In [15]:
# Function to convert to lowercase and remove punctuation
def to_lowercase_and_remove_punctuation(text):
    return text.lower().translate(str.maketrans('', '', string.punctuation))

# Function to display a new question
def display_new_question():
    global question
    question = df.sample(1)
    question_display.value = f"<strong>{question[' Question'].values[0]}</strong>"
    answer_widget.value = ''

# Create a widget to display the question
question_display = widgets.HTML()

# Create a text input widget
answer_widget = widgets.Text(
    value='',
    placeholder='Type your answer',
    description='Your answer:',
    disabled=False
)

# Create a button widget
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click to submit your answer',
    icon='check'
)

# Create a button widget for displaying a new question
new_question_button = widgets.Button(
    description='New Question',
    disabled=False,
    button_style='',
    tooltip='Click to get a new question',
    icon='refresh'
)

# Create an HBox to display the text input widget and the submit button side by side
hbox = widgets.HBox([answer_widget, submit_button])

# Create a VBox to display the question, HBox, and new question button vertically
vbox = widgets.VBox([question_display, hbox, new_question_button])

# Display the VBox
display(vbox)

# Function to check the answer
def check_answer(b):
    answer = answer_widget.value
    if to_lowercase_and_remove_punctuation(answer) == to_lowercase_and_remove_punctuation(question[' Answer'].values[0]):
        print('Correct!')
    else:
        print(f'Incorrect! The correct answer was: {question[" Answer"].values[0]}')

# Attach the check_answer function to the submit button's on_click event
submit_button.on_click(check_answer)

# Attach the display_new_question function to the new question button's on_click event
new_question_button.on_click(lambda b: display_new_question())

# Add an event listener to the answer_widget to submit the answer when Enter is pressed
answer_widget.on_submit(check_answer)

# Display the first question
display_new_question()

Incorrect! The correct answer was: Portugal
Incorrect! The correct answer was: American Ballet Theatre
Incorrect! The correct answer was: English


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.